In [2]:
import gdown
import pandas as pd
import re

In [3]:
# Загружаем таблицу с гугл-диска
url = '15_FfenvHFoJVcPu6gGx9osLn71uDJ-Et'
output = "vacancies.csv"
gdown.download('https://drive.google.com/uc?export=download&id=' + url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?export=download&id=15_FfenvHFoJVcPu6gGx9osLn71uDJ-Et
To: /content/vacancies.csv
100%|██████████| 1.05G/1.05G [00:10<00:00, 104MB/s] 


'vacancies.csv'

In [66]:
df = pd.read_csv("vacancies.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (18,22) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Парсер навыков из вакансий с [гитхаба](https://github.com/keshavbansal015/Job-Description-Parser)

P.S. другие парсеры (в основном платные и для CV):


1.   https://www.maprecruit.ai/parsers - 30 файлов потестила в демо
2.   https://docs.microsoft.com/en-us/connectors/candidatezip/#parse-jd-via-file-content - не смогла зарегистирироваться
3.   http://www.reshufl.com/demo/ - потестила демо



In [ ]:
# Загружаем парсер с гугл-диска
url = '179NC9uvqIbyahO1R3yyiYHV2srB38-5-'
output = "parser.zip"
gdown.download('https://drive.google.com/uc?export=download&id=' + url, output, quiet=False)

In [ ]:
import zipfile
zf = zipfile.ZipFile("parser.zip")
zf.extractall()

In [ ]:
# !pip install spacy
import spacy
sys.path.append(os.path.abspath('./Resume_JD parser(spaCy)/src'))

from func import *
from constants import *
from init_parser import init_parser_func
from tqdm import tqdm
sys.path.append(os.path.abspath('..'))

nlp = spacy.load('en_core_web_sm')

if __name__ == '__main__':)
	matcher = init_parser_func(nlp, './Resume_JD parser(spaCy)/input/Skillset.xlsx', file_type="excel")
	
	final_database = pd.DataFrame()	

	for each in tqdm(range(len(df))):

		text = df.loc[each, 'description']
		application_subject = df.loc[each,'id']
		data = create_profile(nlp,matcher,text,application_subject)
		final_database = final_database.append(data)

	# Saving the database
	final_database.to_csv('./Resume_JD parser(spaCy)/input/Data.csv', index=False)

/content
Success!


100%|██████████| 164916/164916 [2:08:26<00:00, 21.40it/s]


In [ ]:
final_database

,Company/Candidate Name,Skill,Sub-skill,Count
0,47319343,Coding,SQL,1
0,47472541,Analytics,Excel,1
0,46134407,Business,MS Office,1
0,47513275,Business,MS Office,1
0,47727709,Coding,JavaScript,1
...,...,...,...,...
0,46120872,Analytics,Excel,1
0,47121032,Business,MS Office,1
0,47187542,DevOps,MS Word,1
1,47187542,Analytics,Excel,1


# Парсер описания вакансий с помощью регулярных выражений

In [108]:
def JD_new_column(regular: str, new_column: str, df: pd.DataFrame) -> list:
  """
  Создаём метод, который создаёт новый столбец после парсинга
  Аргументы:
  regular - регулярное выражение, согласно которому работает парсер
  new_column - название столбца, который создаётся
  df - датафрейм, который содержит описание вакансий
  Возвращает:
  result - список с результатом работы парсера
  """
  result = []
  df['description'].apply(lambda x: JD(x, regular, result));
  df[f'description_{new_column}'] = pd.DataFrame(result)[0]
  return result


In [107]:
def JD(x: str, regular: str, result: list):
  """
  Создаём метод, который ищет по заданному регулярному выражению в строке
  Аргументы:
  x - строка в датафрейме с описанием вакансий
  regular - регулярное выражение, согласно которому работает парсер
  result - список, куда записывается результат работы метода
  """
  try:
    result.append(re.search(regular, x).group(0))
  except:
    result.append(re.search(regular, x))

In [100]:
result = JD_new_column(r'(?i)Обязанности:.+<strong>|Обязанности:.+\.|чем предстоит заниматься.+\.|задачи.+\.|ищем.{1,15}которые\s{1,}будут.+\.', 'responsipilities', df)

In [101]:
result = JD_new_column(r'(?i)Требования:.+<strong>|Требования:.+\.', 'requirements', df)

In [102]:
result = JD_new_column(r'(?i)Условия:.+<strong>|Условия:.+\.', 'conditions', df)

In [104]:
df[['description_responsipilities', 'description_requirements', 'description_conditions']].head()

,description_responsipilities,description_requirements,description_conditions
0,ОБЯЗАННОСТИ:</strong></p> <ul> <li>Организация...,None,None
1,None,None,None
2,Обязанности:</strong></p> <ul> <li>Сварка кузо...,Требования:</strong> <ul> <li>Опыт работы от 3...,None
3,Обязанности:<br />- Контроль и координация раб...,Требования:<br />- Опыт работы в продажах прив...,"Условия:<br />- Оплата без задержек, премии, б..."
4,Обязанности:</strong> <ul> <li>ведение первичн...,Требования:</strong> <ul> <li>высшее образован...,None


In [111]:
df[['description_responsipilities', 'description_requirements', 'description_conditions']].describe()

,description_responsipilities,description_requirements,description_conditions
count,128272,112464,81226
unique,104119,89801,61375
top,Обязанности:</strong></p><ul><li>Обслуживание ...,Требования: </strong></p><ul><li>Ответственнос...,Условия:</strong></p> <p><strong>
freq,1149,1149,496


In [76]:
# x = pd.DataFrame(result)

In [109]:
# x[0].isnull().sum()

In [40]:
# df.to_csv('dataframe.csv', index=False)